In [1]:
import fitz
import pandas as pd

In [ ]:
doc = fitz.open('X-PRESS MULHACEN H241 MANIFEST.pdf')

In [43]:
total_starts = [] # list of rects for "Shipper :" on all pages
total_stops = [] # list of rects for "Units :" on all pages
total_height = 0.0 # running sum of page heights
total_words = [] # list of all doc words on all pages
total_pods = [] # list of rects for "Port of discharge" on all pages

for page in doc:
    starts = page.searchFor("Shipper :")

    for r in starts:
        total_starts.append(r + (0, total_height, 0, total_height)) # append modif. rect

    stops = page.searchFor("Units :")

    for r in stops:
        total_stops.append(r + (0, total_height, 0, total_height)) # append modif. rect

    wl = page.getTextWords()

    for w in wl:
        total_words.append([w[0], w[1] + total_height, w[2], w[3] + total_height, w[4]])

    pods = page.searchFor('Port of discharge')
    r = 0
    for r in pods:
        total_pods.append(r + (0, total_height, 0, total_height))

    total_height += page.rect.height # to be used for following page


rektangel = total_starts[0] | total_stops[0]
page = doc[0]
def gather_data(x0x1, start, stop):
    
    rektangel = (x0x1[0], start[1], x0x1[1], stop[3])
    get_text = page.get_textbox(rektangel)
    #get_text = [w for w in total_words if fitz.Rect(w[:4]).intersects(rektangel)]

    return get_text

shipper = (33, 181)
goods = (312, 445)

print(gather_data(goods, total_starts[0], total_stops[0]))


20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
20' DC
1   LOT  FERROCHROME
Customs status "N"
Transhipment by  TBN voy. ??? 
ETD XX.10.2021 SYDNEY  ETA 
21.11.2021
Ref. 11221000203  OPS: 
FERROCHROME/Jukka Knuuti
FREIGHT PAYABLE AT As per C/P
MRN: 21FI000000987023E8
